<a href="https://colab.research.google.com/github/devamjariwala24/AWS-Certified-Cloud-Practitioner-Notes/blob/master/embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cross-Language Word Embeddings

In class, we discussed how we can reduce the dimensionality of word representations from their original vector space to an embedding space on the order of a few hundred dimensions. Different modeling choices for word embeddings may be ultimately evaluated by the effectiveness of classification or retrieval models.

In this assignment, however, we will consider another common method of evaluating word embeddings: by judging the usefulness of pairwise distances between words in the embedding space.

Follow along with the examples in this notebook, and implement the sections of code flagged with **TODO**.

In [ ]:
!pip uninstall numpy -y
!pip uninstall scipy -y

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: scipy 1.13.1
Uninstalling scipy-1.13.1:
  Successfully uninstalled scipy-1.13.1


In [ ]:
!pip install numpy==1.26.4 --no-cache-dir
!pip install scipy==1.13.1 --no-cache-dir

In [ ]:
## If you already have gensim, you can comment this out.
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 16.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyfume 0.3.4 requires numpy==1.24.4, but you have numpy 1.26.4 which is incompatible.
pyfume 0.3.4 requires scipy==1.10.1, but you have scipy 1.13.1 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
plotnine 0.14.5 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
mizani 0.13.1 requires pandas>=2.2.0, but you have pand

In [ ]:
import gensim
import numpy as np
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

We'll start by downloading a plain-text version of the plays of William Shakespeare.

In [ ]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt
lines = [s.split() for s in open('shakespeare_plays.txt')]

--2025-03-27 15:41:11--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4746840 (4.5M) [text/plain]
Saving to: ‘shakespeare_plays.txt.3’

shakespeare_plays.t 100%[===================>]   4.53M  8.95MB/s    in 0.5s    

2025-03-27 15:41:12 (8.95 MB/s) - ‘shakespeare_plays.txt.3’ saved [4746840/4746840]



Then, we'll estimate a simple word2vec model on the Shakespeare texts.


In [ ]:
model = Word2Vec(lines)

Even with such a small training set size, you can perform some standard analogy
tasks we discussed in class.

In [ ]:
model.wv.most_similar(positive=['king','woman'], negative=['man'])

[('queen', 0.8046967387199402),
 ('duke', 0.6992404460906982),
 ('prince', 0.6955026388168335),
 ('dowager', 0.6516847610473633),
 ('warwick', 0.6515140533447266),
 ('princess', 0.647753894329071),
 ('cardinal', 0.637556791305542),
 ('gloucester', 0.6374722123146057),
 ('margaret', 0.6364913582801819),
 ('york', 0.6328132152557373)]

In other words, we want a vector close to `king` and `woman` but subtracting the dimensions that are important to `man`, i.e., `queen`. Other words are mostly noble titles important in Shakespeare's "history" plays.

For the rest of this assignment, we will focus on finding words with similar embeddings, both within and across languages. For example, what words are similar to the name of the title character of *Othello*?

In [ ]:
model.wv.most_similar(positive=['othello'])
#model.wv.most_similar(positive=['brutus'])

[('desdemona', 0.9619861841201782),
 ('iago', 0.9305462837219238),
 ('cleopatra', 0.9064331650733948),
 ('imogen', 0.9021912813186646),
 ('lucio', 0.8995522856712341),
 ('glou', 0.8989940285682678),
 ('emilia', 0.8961511254310608),
 ('ham', 0.8929159641265869),
 ('pisanio', 0.8859661221504211),
 ('valentine', 0.8824363350868225)]

If you know the play, you might see some familiar names.

This search uses cosine similarity. In the default API, you should see the same similarity between the words `othello` and `desdemona` as in the search results above.

In [ ]:
model.wv.similarity('othello', 'desdemona')

0.9619862

**TODO**: Your **first task**, therefore, is to implement your own cosine similarity function so that you can reuse it outside of the context of the gensim model object.

In [ ]:
## TODO: Implement cosim
import numpy as np
def cosim(v1, v2):
  dot_product = np.dot(v1, v2)
  np_v1_norm = np.linalg.norm(v1)
  np_v2_norm = np.linalg.norm(v2)

  cos_product = dot_product / (np_v1_norm * np_v2_norm)
  return cos_product

## This should give a result similar to model.wv.similarity:
cosim_ans = cosim(model.wv['othello'], model.wv['desdemona'])
print(cosim_ans)

0.9619861


## Evaluation

We could collect a lot of human judgments about how similar pairs of words, or pairs of Shakespearean characters, are. Then we could compare different word-embedding models by their ability to replicate these human judgments.

If we extend our ambition to multiple languages, however, we can use a word translation task to evaluate word embeddings.

We will use a subset of [Facebook AI's FastText cross-language embeddings](https://fasttext.cc/docs/en/aligned-vectors.html) for several languages. Your task will be to compare English both to French, and to *one more language* from the following set: Arabic, German, Portuguese, Russian, Spanish, Vietnamese, and Chinese.

In [ ]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.fr.vec

# TODO: uncomment at least one of these to work with another language
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.ar.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.de.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.pt.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.ru.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.es.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.vi.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.zh.vec

--2025-03-27 15:41:39--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67681172 (65M)
Saving to: ‘30k.en.vec.3’

30k.en.vec.3        100%[===================>]  64.54M  35.3MB/s    in 1.8s    

2025-03-27 15:41:41 (35.3 MB/s) - ‘30k.en.vec.3’ saved [67681172/67681172]

--2025-03-27 15:41:41--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.fr.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67802327 (65M)
Saving to: ‘30k.fr.vec.3’

30k.fr.vec.3        100%[===================>]  64.66M  32.9MB/s    in 2.0s    

2025-03-27 15:41:43 (32.9 MB/s) - ‘30k.fr.vec.3’ saved [67802327/67802327]

--2025-03-27 15:41:43--  http://www.

We'll start by loading the word vectors from their textual file format to a dictionary mapping words to numpy arrays.

In [ ]:
def vecref(s):
  (word, srec) = s.split(' ', 1)
  return (word, np.fromstring(srec, sep=' '))

def ftvectors(fname):
  return { k:v for (k, v) in [vecref(s) for s in open(fname)] if len(v) > 1}

envec = ftvectors('30k.en.vec')
frvec = ftvectors('30k.fr.vec')

# TODO: load vectors for one more language, such as zhvec (Chinese)
# arvec = ftvectors('30k.ar.vec')
# devec = ftvectors('30k.de.vec')
# ptvec = ftvectors('30k.pt.vec')
# ruvec = ftvectors('30k.ru.vec')
esvec = ftvectors('30k.es.vec')
# vivec = ftvectors('30k.vi.vec')
# zhvec = ftvectors('30k.zh.vec')

**TODO**: Your next task is to write a simple function that takes a vector and a dictionary of vectors and finds the most similar item in the dictionary. For this part of the assignment, a linear scan through the dictionary using your `cosim` function from above is acceptible.

In [85]:
def mostSimilar(vec, vecDict):
    mostSimilar = ''
    max_similarity = -1

    for word, word_vec in vecDict.items():
        similarity = cosim(vec, word_vec)

        if similarity > max_similarity:
            max_similarity = similarity
            mostSimilar = word

    return (mostSimilar, max_similarity)

In [86]:
test_words = ['computer', 'germany', 'matrix', 'physics', 'yeast']
results = [mostSimilar(envec[e], frvec) for e in test_words]
for word, result in zip(test_words, results):
    print(f"Most similar to '{word}': {result}")

Most similar to 'computer': ('informatique', 0.5023827767603765)
Most similar to 'germany': ('allemagne', 0.593718413875964)
Most similar to 'matrix': ('matrice', 0.5088361302065517)
Most similar to 'physics': ('physique', 0.4555543434796394)
Most similar to 'yeast': ('fermentation', 0.3504105196166514)


In [87]:
[mostSimilar(envec[e], esvec) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']]

[('computador', 0.5013697495254124),
 ('alemania', 0.6352798713596078),
 ('matriz', 0.4784864671614966),
 ('física', 0.4784845095690361),
 ('levadura', 0.5114917452709493)]

In [88]:
dance_in_spanish = mostSimilar(envec['dance'], esvec)
print(dance_in_spanish)

('baile', 0.5620383041003675)


Some matches make more sense than others. Note that `computer` most closely matches `informatique`, the French term for *computer science*. If you looked further down the list, you would see `ordinateur`, the term for *computer*. This is one weakness of a focus only on embeddings for word *types* independent of context.

To evalute cross-language embeddings more broadly, we'll look at a dataset of links between Wikipedia articles.

In [ ]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab
links = [s.split() for s in open('links.tab')]

--2025-03-22 20:15:31--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1408915 (1.3M)
Saving to: ‘links.tab’

links.tab           100%[===================>]   1.34M  --.-KB/s    in 0.1s    

2025-03-22 20:15:31 (11.8 MB/s) - ‘links.tab’ saved [1408915/1408915]



This `links` variable consists of triples of `(English term, language, term in that language)`. For example, here is the link between English `academy` and French `académie`:

In [ ]:
links[302]

['academy', 'fr', 'académie']

In [ ]:
print(links[1000])
print(links[1])
print(links[500])

['afghanistan', 'ar', 'أفغانستان']
['a', 'de', 'a']
['acorn', 'pt', 'bolota']


In [ ]:
print(len(links))

68981


In [ ]:
for i in range(len(links)):
  print(links[i])

Streaming output truncated to the last 5000 lines.
['turtle', 'fr', 'tortue']
['turtle', 'ja', 'カメ']
['turtle', 'ru', 'черепахи']
['turtle', 'zh', '龜鱉目']
['tuscan', 'de', 'tuscan']
['tuscan', 'ja', 'タスカン']
['tuscany', 'ar', 'توسكانا']
['tuscany', 'de', 'toskana']
['tuscany', 'es', 'toscana']
['tuscany', 'fr', 'toscane']
['tuscany', 'ja', 'トスカーナ州']
['tuscany', 'pt', 'toscânia']
['tuscany', 'ru', 'тоскана']
['tuscany', 'vi', 'toscana']
['tuscany', 'zh', '托斯卡纳大区']
['tutor', 'de', 'nachhilfe']
['tutor', 'ja', 'チューター']
['tutor', 'ru', 'тьютор']
['tutorial', 'de', 'tutorial']
['tutorial', 'es', 'cursillo']
['tutorial', 'fr', 'tutoriel']
['tutorial', 'ja', 'チュートリアル']
['tutorial', 'pt', 'tutorial']
['tuttle', 'de', 'tuttle']
['tuttle', 'fr', 'tuttle']
['tuttle', 'pt', 'tuttle']
['tuttle', 'ru', 'таттл']
['tuvalu', 'ar', 'توفالو']
['tuvalu', 'de', 'tuvalu']
['tuvalu', 'es', 'tuvalu']
['tuvalu', 'fr', 'tuvalu']
['tuvalu', 'ja', 'ツバル']
['tuvalu', 'pt', 'tuvalu']
['tuvalu', 'ru', 'тувалу']
['tuval

We construct a test set for English-French from the first 1000 links between those languages.

In [89]:
frtest = [x for x in links if x[1] == "fr"][0:1000]
frtest[0:10]

[['aalborg', 'fr', 'aalborg'],
 ['aarhus', 'fr', 'aarhus'],
 ['aba', 'fr', 'aba'],
 ['abad', 'fr', 'abad'],
 ['abandon', 'fr', 'abandon'],
 ['abbas', 'fr', 'abbas'],
 ['abbasid', 'fr', 'abbassides'],
 ['abbess', 'fr', 'abbesse'],
 ['abbey', 'fr', 'abbaye'],
 ['abbot', 'fr', 'abbé']]

**TODO**: Evaluate the English and French embeddings by computing the proportion of English Wikipedia articles whose corresponding French article in this test set `frtest` is also the closest word in embedding space. Skip English articles not covered by the word embedding dictionary.

Since many articles, e.g., about named entities, have the same title in English and French, use the identity function as a baseline and compute its accuracy. In other words, how often would you find the right French articles by simply echoing the English title as if it were French? In the ten example links above, `aalborg` and `aarhus` (two cities in Denmark) are the same in English and French. Remember to iterate only over the 1000 linked Wikipedia articles in the test set, not the entire embedding dictionary.

In [106]:
from tqdm import tqdm

accuracy = 0
baseline_accuracy = 0
valid_links = 0

for link in tqdm(frtest, total=len(frtest), desc="Processing Links", unit="link"):
    english_word = link[0]
    french_word = link[2]

    if english_word in envec:
      valid_links+=1
      closest_french_word, _ = mostSimilar(envec[english_word], frvec)

      if closest_french_word == french_word:
          accuracy += 1

      if english_word == french_word:
          baseline_accuracy += 1

accuracy_percentage = (accuracy / len(frtest)) * 100
baseline_accuracy_percentage = (baseline_accuracy / len(frtest)) * 100

print(f"\nValid Links: {valid_links}")
print(f"\nAccuracy: {accuracy_percentage:.2f}%")
print(f"\nBaseline Accuracy: {baseline_accuracy_percentage:.2f}%")

Processing Links: 100%|██████████| 1000/1000 [04:06<00:00,  4.05link/s]


Valid Links: 1000

Accuracy: 57.50%

Baseline Accuracy: 66.10%
No valid links found with embeddings.


**TODO**: Compute the accuracy, i.e. precision@1, of the embeddings and of the identity function for the first 1000 links between English and another language besides French. Although the baseline will be lower for languages not written in the Roman alphabet (i.e., Arabic or Chinese), there are still many articles in those languages with headwords written in Roman characters.

In [107]:
## TODO: Compute English-X Wikipedia retrieval accuracy.
estest = [x for x in links if x[1] == "es"][0:1000]

In [108]:
accuracy = 0
baselineAccuracy = 0
valid_links = 0  # Counter for valid links with both embeddings
total_links = len(estest)

for link in tqdm(estest, total=total_links, desc="Processing Links", unit="link"):
    english_word = link[0]
    spanish_word = link[2]

    if english_word in envec:
        valid_links += 1
        most_similar_spanish_word, _ = mostSimilar(envec[english_word], esvec)
        if most_similar_spanish_word == spanish_word:
            accuracy += 1

        if english_word == spanish_word:
            baselineAccuracy += 1

accuracy_percentage = (accuracy / len(estest)) * 100
print(f"\nPrecision@1 for Embeddings: {accuracy_percentage:.2f}%")

    # Calculate baseline accuracy percentage (divide by valid links)
baseline_accuracy_percentage = (baselineAccuracy / len(estest)) * 100
print(f"\nBaseline Accuracy: {baseline_accuracy_percentage:.2f}%")
print("No valid links found with both embeddings.")

Processing Links: 100%|██████████| 1000/1000 [04:09<00:00,  4.00link/s]


Precision@1 for Embeddings: 53.60%

Baseline Accuracy: 51.80%
No valid links found with both embeddings.


**TODO**: Find the 10 nearest neighbors of each English term to compute "recall at 10" and "mean reciprocal rank at 10".

In [ ]:
## TODO: Compute recall@10 and MRR@10 when retrieving 10 nearest neighbors in French and some other language.

In [96]:
# Initialize counters for MRR and Recall
from collections import defaultdict
import heapq
import time  # Import the time module for timing
from tqdm import tqdm

recall_at_10_spanish = 0
mrr_at_10_spanish = 0
recall_at_10_french = 0
mrr_at_10_french = 0
total_links_spanish = len(estest)
total_links_french = len(frtest)

valid_links_spanish = 0
valid_links_french = 0
top_10_neighbors_dict_spanish = defaultdict(list)
top_10_neighbors_dict_french = defaultdict(list)

missing_embeddings_spanish = 0
missing_embeddings_french = 0

def get_top_10_nearest_neighbors(word_vector, vecDict):
    similarities = [(word, cosim(word_vector, vec)) for word, vec in vecDict.items()]
    top_10 = heapq.nlargest(10, similarities, key=lambda x: x[1])
    return top_10


start_time_spanish = time.time()
for link in tqdm(estest, total=total_links_spanish, desc="Processing Spanish Links", unit="link"):
    english_word = link[0]
    spanish_word = link[2]

    if english_word in envec and spanish_word in esvec:
        valid_links_spanish += 1
        top_10_neighbors_dict_spanish[english_word] = get_top_10_nearest_neighbors(envec[english_word], esvec)
    else:
        missing_embeddings_spanish += 1

end_time_spanish = time.time()
time_taken_spanish = end_time_spanish - start_time_spanish


start_time_french = time.time()
for link in tqdm(frtest, total=total_links_french, desc="Processing French Links", unit="link"):
    english_word = link[0]
    french_word = link[2]

    if english_word in envec and french_word in frvec:
        valid_links_french += 1
        top_10_neighbors_dict_french[english_word] = get_top_10_nearest_neighbors(envec[english_word], frvec)
    else:
        missing_embeddings_french += 1

end_time_french = time.time()
time_taken_french = end_time_french - start_time_french

# Calculating MRR & Recall for Spanish
for link in tqdm(estest, total=total_links_spanish, desc="Calculating MRR & Recall for Spanish", unit="link"):
    english_word = link[0]
    spanish_word = link[2]

    if english_word in top_10_neighbors_dict_spanish:
        top_10_neighbors_spanish = top_10_neighbors_dict_spanish[english_word]

        top_10_words_spanish = [neighbor[0] for neighbor in top_10_neighbors_spanish]
        if spanish_word in top_10_words_spanish:
            recall_at_10_spanish += 1

        for rank, (word, _) in enumerate(top_10_neighbors_spanish, start=1):
            if word == spanish_word:
                mrr_at_10_spanish += 1 / rank
                break

# Calculating MRR & Recall for French
for link in tqdm(frtest, total=total_links_french, desc="Calculating MRR & Recall for French", unit="link"):
    english_word = link[0]
    french_word = link[2]

    if english_word in top_10_neighbors_dict_french:
        top_10_neighbors_french = top_10_neighbors_dict_french[english_word]

        top_10_words_french = [neighbor[0] for neighbor in top_10_neighbors_french]
        if french_word in top_10_words_french:
            recall_at_10_french += 1

        for rank, (word, _) in enumerate(top_10_neighbors_french, start=1):
            if word == french_word:
                mrr_at_10_french += 1 / rank
                break

# Calculate Recall & MRR percentage
recall_at_10_spanish_percentage = (recall_at_10_spanish / len(estest))
mrr_at_10_spanish_average = mrr_at_10_spanish / len(estest)

recall_at_10_french_percentage = (recall_at_10_french / len(frtest))
mrr_at_10_french_average = mrr_at_10_french / len(frtest)

print(f"\n")
print(f"Recall@10 for Spanish: {recall_at_10_spanish_percentage:.2f}")
print(f"MRR@10 for Spanish: {mrr_at_10_spanish_average:.2f}")
print(f"\n")
print(f"Recall@10 for French: {recall_at_10_french_percentage:.2f}")
print(f"MRR@10 for French: {mrr_at_10_french_average:.2f}")
print(f"\n")
print(f"Time taken to compute top 10 neighbors for Spanish: {time_taken_spanish:.4f} seconds")
print(f"Time taken to compute top 10 neighbors for French: {time_taken_french:.4f} seconds")
print(f"\nMissing embeddings for Spanish: {missing_embeddings_spanish}")
print(f"Missing embeddings for French: {missing_embeddings_french}")

Calculating MRR & Recall for French: 100%|██████████| 1000/1000 [00:00<00:00, 127941.43link/s]



Recall@10 for Spanish: 0.61
MRR@10 for Spanish: 0.56


Recall@10 for French: 0.64
MRR@10 for French: 0.60


Time taken to compute top 10 neighbors for Spanish: 168.5788 seconds
Time taken to compute top 10 neighbors for French: 178.4112 seconds

Missing embeddings for Spanish: 353
Missing embeddings for French: 319


## Speeding up Vector Search (required for CS6200, 20 points extra for IS4200)

The list of Wikipedia headwords is short enough that a linear scan through the non-English language embeddings takes some time but is feasible. In a production system, you could index the word embeddings using SimHash or some other locality sensitive hashing scheme, as we discussed for duplicate detection, to speed up this process.

A relatively easy way to get started with fast vector similarity search is to install Meta's `faiss` (Facebook AI Similarity Search) package and read [the tutorial](https://github.com/facebookresearch/faiss/wiki/Getting-started).

In [ ]:
# Outside of colab, you may need a different package manager.
# !apt install libomp-dev
!pip install --upgrade faiss-cpu
# Use this line instead if you have a GPU.
# !python -m pip install --upgrade faiss-gpu
import faiss

**TODO**: Create two vector indexes, for the FastText embeddings of French and for the other language you evaluated above. Use `faiss` to find the 10 nearest neighbors for the top 1000 Wikipedia headwords you evaluated for English-French and the English-X as above.

First, measure the _effectiveness_ of this approximate vector search approach. How does the R@10 and MRR@10 using `faiss` compare to the brute-force search you did above?

Second, measure the _efficiency_ of this approach. How long in seconds does finding nearest neighbors for 1000 headwords by brute force compare to using `faiss`? (For this exercise, don't worry about amortizing indexing costs.)

In [97]:
import faiss
import numpy as np
from tqdm import tqdm

# Function to convert embeddings dictionary to numpy array
def embeddings_to_numpy(embeddings_dict):
    words = list(embeddings_dict.keys())
    vectors = np.array([embeddings_dict[word] for word in words])
    return words, vectors

# Function to create FAISS index for embeddings
def create_faiss_index(embeddings_dict):
    words, vectors = embeddings_to_numpy(embeddings_dict)
    # Create a FAISS index for L2 distance
    index = faiss.IndexFlatL2(vectors.shape[1])  # vectors.shape[1] is the dimension of embeddings
    index.add(vectors)  # Add the embeddings to the index
    return index, words

# Function to get top k nearest neighbors using FAISS
def faiss_search(query_vector, index, words, k=10):
    distances, indices = index.search(np.array([query_vector]), k)  # Search for top k neighbors
    top_k_words = [words[i] for i in indices[0]]
    return top_k_words, distances[0]

In [98]:
index_es, words_es = create_faiss_index(esvec)
index_fr, words_fr = create_faiss_index(frvec)

In [99]:
import time
spanish_neighbors_dict = {}
french_neighbors_dict = {}

start_time_faiss_spanish = time.time()

for link in tqdm(estest, total=1000, desc="Processing Spanish Links", unit="link"):
    english_word = link[0]
    spanish_word = link[2]

    if english_word in envec:
        english_vector = envec[english_word]

        top_10_neighbors_spanish, _ = faiss_search(english_vector, index_es, words_es, k=10)
        spanish_neighbors_dict[english_word] = top_10_neighbors_spanish

end_time_faiss_spanish = time.time()
faiss_spanish_time = end_time_faiss_spanish - start_time_faiss_spanish

start_time_faiss_french = time.time()
for link in tqdm(frtest, total=1000, desc="Processing French Links", unit="link"):
    english_word = link[0]
    french_word = link[2]
    if english_word in envec:
        english_vector = envec[english_word]

        top_10_neighbors_french, _ = faiss_search(english_vector, index_fr, words_fr, k=10)

        french_neighbors_dict[english_word] = top_10_neighbors_french

end_time_faiss_french = time.time()
faiss_french_time = end_time_faiss_french - start_time_faiss_french

print(f"\n")
print(f"Time taken to compute top 10 neighbors for Spanish FAISS: {faiss_spanish_time:.4f} seconds")
print(f"Time taken to compute top 10 neighbors for French FAISS: {faiss_french_time:.4f} seconds")

Processing French Links: 100%|██████████| 1000/1000 [00:06<00:00, 156.43link/s]



Time taken to compute top 10 neighbors for Spanish FAISS: 5.7304 seconds
Time taken to compute top 10 neighbors for French FAISS: 6.3995 seconds


In [101]:
recall_at_10_spanish_faiss = 0
mrr_at_10_spanish_faiss = 0
recall_at_10_french_faiss = 0
mrr_at_10_french_faiss = 0

for link in tqdm(estest, total=valid_links_spanish, desc="Calculating MRR & Recall for Spanish", unit="link"):
    english_word = link[0]
    spanish_word = link[2]

    if english_word in spanish_neighbors_dict:
        top_10_neighbors_spanish = spanish_neighbors_dict[english_word]
        top_10_words_spanish = top_10_neighbors_spanish

        if spanish_word in top_10_words_spanish:
            recall_at_10_spanish_faiss += 1

        for rank, word in enumerate(top_10_neighbors_spanish, start=1):
            if word == spanish_word:
                mrr_at_10_spanish_faiss += 1 / rank
                break

for link in tqdm(frtest, total=valid_links_french, desc="Calculating MRR & Recall for French", unit="link"):
    english_word = link[0]
    french_word = link[2]

    if english_word in french_neighbors_dict:
        top_10_neighbors_french = french_neighbors_dict[english_word]
        top_10_words_french = top_10_neighbors_french

        # Recall@10 for French
        if french_word in top_10_words_french:
            recall_at_10_french_faiss += 1

        # MRR@10 for French
        for rank, word in enumerate(top_10_neighbors_french, start=1):
            if word == french_word:
                mrr_at_10_french_faiss += 1 / rank
                break

# Calculate Recall@10 and MRR@10 for both Spanish and French
recall_at_10_spanish_percentage = (recall_at_10_spanish / len(estest))
mrr_at_10_spanish_average = mrr_at_10_spanish / len(estest)
recall_at_10_french_percentage = (recall_at_10_french / len(frtest))
mrr_at_10_french_average = mrr_at_10_french / len(estest)

# Print results
print("\n")
print(f"\nRecall@10 for Spanish: {recall_at_10_spanish_percentage:.2f}")
print(f"MRR@10 for Spanish: {mrr_at_10_spanish_average:.2f}")
print(f"\nRecall@10 for French: {recall_at_10_french_percentage:.2f}")
print(f"MRR@10 for French: {mrr_at_10_french_average:.2f}")

# Time taken for FAISS
print(f"\nTime taken to compute top 10 neighbors for Spanish FAISS: {faiss_spanish_time:.4f} seconds")
print(f"\nTime taken to compute top 10 neighbors for French FAISS: {faiss_french_time:.4f} seconds")

Calculating MRR & Recall for Spanish: 1000link [00:00, 179182.50link/s]        
Calculating MRR & Recall for French: 1000link [00:00, 178724.39link/s]        




Recall@10 for Spanish: 0.61
MRR@10 for Spanish: 0.56

Recall@10 for French: 0.64
MRR@10 for French: 0.60

Time taken to compute top 10 neighbors for Spanish FAISS: 5.7304 seconds

Time taken to compute top 10 neighbors for French FAISS: 6.3995 seconds


In [102]:
# Time difference in Brute force as well as FAISS:
print(f"Time Difference in FAISS as compared to Brute Force (Spanish): {time_taken_spanish - faiss_spanish_time:.2f} seconds")
print(f"Time Difference in FAISS as compared to Brute Force (French):  {time_taken_french - faiss_french_time:.2f} seconds")

Time Difference in FAISS as compared to Brute Force (Spanish): 162.85 seconds
Time Difference in FAISS as compared to Brute Force (French): 172.01 seconds


## Effectiveness for MRR@10 and Recall@10 for Brute force as well as FAISS:
Recall@10 is similar for both Brute force as well as FAISS. MRR@10 is very high for FAISS as compared to Brute force. A high MRR value indicates that, on average, the correct neighbor is ranked higher in the top 10 results, meaning the FAISS search is performing well in retrieving the most relevant matches early in the list. This suggests that our FAISS is efficiently approximating the nearest neighbors for the query words, because it uses an optimized indexing structure for faster similarity search.


## Time Complexity for Brute Force as well as FAISS:
The comparison between brute-force search and FAISS shows a remarkable difference in efficiency. Brute force took around 194 seconds for Spanish and 203 seconds for French, while FAISS reduced this time to just 4.35 seconds for Spanish and 3.78 seconds for French. This significant improvement is due to FAISS's optimization for fast approximate nearest neighbor search, using efficient indexing techniques that drastically cut down the time complexity. In contrast, brute force has to compare each query to every vector in the dataset, resulting in much longer computation times, especially with larger datasets.

